In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os.path
import pathlib

In [2]:
cwd = pathlib.Path().resolve()
src = cwd.parent
data = src.parent.parent.parent

data_path = os.path.join(data, 'OneDrive - Delft University of Technology', 'TU Delft', 'Master ENVM', 'MDP', 'Model', 'Data', 'TAHMO')

In [10]:
data_files = glob.glob(os.path.join(data_path, '*.csv'))

dataframes = {}

for file in data_files:
    station_name = os.path.splitext(os.path.basename(file))[0].split('_')[0]
    df = pd.read_csv(file, index_col = 0, sep =',', parse_dates = True)
    dataframes[station_name] = df

dataframes

In [ ]:
dataframes.index = 

In [4]:
def Extra_rad(Tmax, Tmin, df, lat):
    "Calculation of Extraterrestrial radiation"
    G = 0.0820 #* 10 ** 6 # J/m^2/min  --> Constant value

    # Calculation of the radian location of the station
    phi = np.pi / 180 * lat

    # Calculation of the number of the day in a year
    J = df.index.dayofyear.values  # Extract day of the year directly from the index

    # Calculation of the extraterrestrial radiation: Ra
    dr = 1 + 0.033 * np.cos(2 * np.pi * J / 365)
    delta = 0.409 * np.sin((2 * np.pi * J / 365) - 1.39)
    w = np.arccos(- np.tan(phi) * np.tan(delta))
    Ra = ((24 * 60) / np.pi) * G * dr * (w * np.sin(phi) * np.sin(delta) + np.cos(phi) * np.cos(delta) * np.sin(w)) #* 10 ** - 6

    # Create a DataFrame with Ra values and corresponding timestamps
    Ra_df = pd.DataFrame({'Ra': Ra}, index=df.index)

    return Ra_df